In [7]:
import requests

In [11]:
with open('token.txt', 'r') as file:
    TOKEN = file.read().strip()
COUNTRY_ID = 21 # Australia

In [ ]:
headers = {
    "Authorization": f"FPDAPI SubscriberToken={TOKEN}",
    "Content-Type": "application/json"
}

In [12]:
# 1. Find Cairns GeoRegionId
response_regions = requests.get(
    "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber/GetCountryGeographicRegions",
    headers=headers,
    params={"countryId": COUNTRY_ID}
)
geo_regions = response_regions.json()["GeographicRegions"]

cairns_region_id = next(
    region["GeoRegionId"]
    for region in geo_regions
    if region["Name"] == "Cairns" and region["GeoRegionLevel"] == 2
)
print(f"Cairns GeoRegionId = {cairns_region_id}")

Cairns GeoRegionId = 16


In [15]:
# 2. Get all prices for Cairns
response_prices = requests.get(
    "https://fppdirectapi-prod.fuelpricesqld.com.au/Price/GetSitesPrices",
    headers=headers,
    params={
        "countryId": COUNTRY_ID,
        "geoRegionLevel": 2,
        "geoRegionId": cairns_region_id
    }
)
site_prices = response_prices.json()["SitePrices"]

In [ ]:
# 3. Get site details for Cairns (site names, addresses, etc)
response_sites = requests.get(
    "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber/GetFullSiteDetails",
    headers=headers,
    params={
        "countryId": COUNTRY_ID,
        "geoRegionLevel": 2,
        "geoRegionId": cairns_region_id
    }
)
site_details = response_sites.json()["S"]

# Map: site_id → site_name
site_code_to_site_data = {
    int(site["S"]): site["N"] for site in site_details
}


In [ ]:

site_entries = response_sites.json()["S"]

site_code_to_site_data = {}
for entry in site_entries:
    site_id = int(entry["S"])
    site_code_to_site_data[site_id] = {
        "name": entry.get("N", ""),
        "address": entry.get("A", ""),
        "brand_id": int(entry["B"]) if entry.get("B") is not None else None,
        "postcode": entry.get("P", ""),
        "geo_region_id_level_1": entry.get("G1"),
        "geo_region_id_level_2": entry.get("G2"),
        "geo_region_id_level_3": entry.get("G3"),
        "geo_region_id_level_4": entry.get("G4"),
        "geo_region_id_level_5": entry.get("G5"),
        "latitude": float(entry["Lat"]) if entry.get("Lat") is not None else None,
        "longitude": float(entry["Lng"]) if entry.get("Lng") is not None else None,
        "google_place_id": entry.get("GPI", ""),
        "last_modified": entry.get("M"),  # ISO timestamp or None
    }

61401973 {'name': 'United Gordonvale', 'address': '68 Riverstone Road', 'brand_id': 23, 'postcode': '4865', 'geo_region_id_level_1': 182624492, 'geo_region_id_level_2': 16, 'geo_region_id_level_3': 1, 'geo_region_id_level_4': 0, 'geo_region_id_level_5': 0, 'latitude': -17.099007, 'longitude': 145.779894, 'google_place_id': 'ChIJvepmggmE2GoROzLfENNmBTY', 'last_modified': '2019-07-15T05:32:17.627'}


In [28]:
site_code_to_site_data

{61401973: {'name': 'United Gordonvale',
  'address': '68 Riverstone Road',
  'brand_id': 23,
  'postcode': '4865',
  'geo_region_id_level_1': 182624492,
  'geo_region_id_level_2': 16,
  'geo_region_id_level_3': 1,
  'geo_region_id_level_4': 0,
  'geo_region_id_level_5': 0,
  'latitude': -17.099007,
  'longitude': 145.779894,
  'google_place_id': 'ChIJvepmggmE2GoROzLfENNmBTY',
  'last_modified': '2019-07-15T05:32:17.627'},
 61401991: {'name': 'BP Edmonton',
  'address': '3/167 Bruce Hwy',
  'brand_id': 5,
  'postcode': '4869',
  'geo_region_id_level_1': 1386,
  'geo_region_id_level_2': 16,
  'geo_region_id_level_3': 1,
  'geo_region_id_level_4': 0,
  'geo_region_id_level_5': 0,
  'latitude': -17.017429,
  'longitude': 145.743907,
  'google_place_id': 'ChIJ5XNUMg5ieGkRzilzX_ictH0',
  'last_modified': '2019-07-15T05:32:17.627'},
 61402009: {'name': 'Ampol Mossman',
  'address': '67 Alchera',
  'brand_id': 3421066,
  'postcode': '4873',
  'geo_region_id_level_1': 1802,
  'geo_region_id_le

In [33]:
# plot the locations of the fuel stations on a map
import folium
cairns_map = folium.Map(location=[-16.9186, 145.7781], zoom_start=12)

# Add markers for each fuel station
for site_id, site_data in site_code_to_site_data.items():
    folium.Marker(
        location=[site_data["latitude"], site_data["longitude"]],
        popup=site_data["name"],
    ).add_to(cairns_map)

cairns_map.save("cairns_fuel_stations_map.html")
# show the map
cairns_map
